In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, json, shutil
from glob import glob
base_dir="/content/drive/MyDrive/ML-Task"

In [3]:
file_path =glob("/content/drive/MyDrive/ML-Task/KoLa Dataset/BB/*.josn")

In [5]:
# appedn all files together
bboxes=[]
for file in file_path:
  with open(file, 'rb') as f:
    box=json.load(f)
    bboxes.append(box)

In [18]:
# create a directory for labels
labels = os.path.join(base_dir, 'finale_labels')
os.makedirs(labels, exist_ok=True)

In [24]:
for i in range(len(bboxes)):
  image_height =bboxes[i]['imageHeight']
  image_width=bboxes[i]['imageWidth']
  image_id =os.path.splitext(os.path.basename(bboxes[i]['imagePath']))[0]
  image_points =[]
  for shape in bboxes[i]['shapes']:
    label =shape['label']
    points=shape['points']
    x_min =points[0][0]/image_width
    y_min =points[0][1]/image_height
    x_max =points[1][0]/image_width
    y_max =points[1][1]/image_height

    #find out the mid and height width
    x_mid = (x_max + x_min)/2
    y_mid =(y_max - y_min)/2

    w= x_max - x_min
    h= y_max - y_min
    image_points.append(f'{label} {x_mid} {y_mid} {w} {h}')


  with open(f'/content/drive/MyDrive/ML-Task/finale_labels/{image_id}.txt', 'w') as txt_file:
    txt_file.write('\n'.join(image_points))


In [25]:
image_paths =glob("/content/drive/MyDrive/ML-Task/KoLa Dataset/Kola/*")
label_paths =glob("/content/drive/MyDrive/ML-Task/finale_labels/*.txt")

In [29]:
len(image_paths), len(label_paths)

(167, 162)

In [30]:
valid_image_paths = []
valid_label_paths = []
for image_path in image_paths:
    image_filename=os.path.splitext(os.path.basename(image_path))[0]
    label_filename = image_filename +'.txt'
    label_path = os.path.join("/content/drive/MyDrive/ML-Task/finale_labels", label_filename)
    if label_path in label_paths:
        valid_image_paths.append(image_path)
        valid_label_paths.append(label_path)


In [32]:
len(valid_image_paths), len(valid_label_paths)

(163, 163)

In [34]:
from sklearn.model_selection import KFold

In [38]:
n_splits = 3  # Specify the number of folds
test_size = 30  # Specify the number of test samples for each fold


test_dir_images = os.path.join(base_dir, 'koala/images/test')
test_dir_labels = os.path.join(base_dir, 'koala/labels/test')
os.makedirs(test_dir_images, exist_ok=True)
os.makedirs(test_dir_labels, exist_ok=True)
kf=KFold(n_splits=n_splits, shuffle=True, random_state=42)
data=list(zip(valid_image_paths,valid_label_paths))

In [42]:
for fold, (train_index, test_index) in enumerate(kf.split(data)):
    train_data = [data[i] for i in train_index]
    test_data = [data[i] for i in test_index[:test_size]]

    remaining_train_index = test_index[test_size:]
    remaining_train_data = [data[i] for i in remaining_train_index]
    train_data.extend(remaining_train_data)

    # Create directories for training
    train_dir_images = os.path.join(base_dir, f'koala/images/fold{fold + 1}')
    train_dir_labels = os.path.join(base_dir, f'koala/labels/fold{fold + 1}')

    os.makedirs(train_dir_images, exist_ok=True)
    os.makedirs(train_dir_labels, exist_ok=True)

    # Copy train data
    for img_path, txt_path in train_data:
        shutil.copyfile(img_path, os.path.join(train_dir_images, os.path.basename(img_path)))
        shutil.copyfile(txt_path, os.path.join(train_dir_labels, os.path.basename(txt_path)))

# Copy test data directly under 'images/test'
for img_path, txt_path in test_data:
    shutil.copyfile(img_path, os.path.join(test_dir_images, os.path.basename(img_path)))
    shutil.copyfile(txt_path, os.path.join(test_dir_labels, os.path.basename(txt_path)))

In [41]:
class_name = "koala"

file_name = "koala/classes.txt"
file_path = os.path.join(base_dir, file_name)
with open(file_path, 'w') as file:
    file.write(class_name)

In [ ]:
# dataset Preparation Complete.